In [2]:
# unzip dataset_letras.zip
!unzip dataset_letras_final.zip

Streaming output truncated to the last 5000 lines.
 extracting: dataset_letras3/Q/Q_106.png  
 extracting: dataset_letras3/Q/Q_107.png  
 extracting: dataset_letras3/Q/Q_108.png  
 extracting: dataset_letras3/Q/Q_109.png  
 extracting: dataset_letras3/Q/Q_11.png  
 extracting: dataset_letras3/Q/Q_110.png  
 extracting: dataset_letras3/Q/Q_111.png  
 extracting: dataset_letras3/Q/Q_112.png  
 extracting: dataset_letras3/Q/Q_113.png  
 extracting: dataset_letras3/Q/Q_114.png  
 extracting: dataset_letras3/Q/Q_115.png  
 extracting: dataset_letras3/Q/Q_116.png  
 extracting: dataset_letras3/Q/Q_117.png  
 extracting: dataset_letras3/Q/Q_118.png  
 extracting: dataset_letras3/Q/Q_119.png  
 extracting: dataset_letras3/Q/Q_12.png  
 extracting: dataset_letras3/Q/Q_120.png  
 extracting: dataset_letras3/Q/Q_121.png  
 extracting: dataset_letras3/Q/Q_122.png  
 extracting: dataset_letras3/Q/Q_123.png  
 extracting: dataset_letras3/Q/Q_124.png  
 extracting: dataset_letras3/Q/Q_125.png  
 extr

In [3]:
!pip install WandB
import wandb

## Login
wandb.login(key='a14c6a2ec25620e6e2047f787c8dbe5d7710eaef')

## Inicializa WandB
wandb.init(project="repte1_psiv", entity="andreu-mir")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: andreu-mir. Use `wandb login --relogin` to force relogin


In [5]:
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import wandb

# Iniciar WandB
wandb.init(project="SVM_CrossValidation_Letters")

# Definición de parámetros
batch_size = 32
img_height, img_width = 64, 40  # Tamaño de las imágenes
num_folds = 5  # Número de pliegues (folds) para la validación cruzada

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Redimensionar las imágenes
    transforms.Grayscale(num_output_channels=1),  # Escala de grises
    transforms.ToTensor(),  # Convertir imágenes a tensores
    transforms.Normalize((0.5,), (0.5,)),  # Normalizar las imágenes
])

# Cargar el dataset
base_dir = 'dataset_letras3'
dataset = datasets.ImageFolder(root=base_dir, transform=transform)

# Función para extraer características planas de las imágenes
def extract_features(loader):
    features = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1)  # Flatten each image to a vector
        features.append(images.numpy())
        labels.append(lbls.numpy())

    return np.vstack(features), np.hstack(labels)

# Configurar el objeto KFold
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop a través de los pliegues de validación cruzada
for fold, (train_indices, val_indices) in enumerate(kfold.split(range(len(dataset)))):
    print(f'Fold {fold + 1}/{num_folds}')

    # Crear subconjuntos de entrenamiento y validación
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)

    # Crear DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Extraer características de los conjuntos de entrenamiento y validación
    train_features, train_labels = extract_features(train_loader)
    val_features, val_labels = extract_features(val_loader)

    # Definir el modelo SVM con un pipeline de estandarización
    svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))

    # Entrenar el modelo SVM
    svm_model.fit(train_features, train_labels)

    # Evaluar el modelo en el conjunto de validación
    val_predictions = svm_model.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)
    precision = precision_score(val_labels, val_predictions, average='weighted')
    recall = recall_score(val_labels, val_predictions, average='weighted')
    f1 = f1_score(val_labels, val_predictions, average='weighted')

    # Imprimir resultados del fold
    print(f"Fold {fold + 1} - Accuracy: {accuracy * 100:.2f}%")
    print(f"Fold {fold + 1} - Precision: {precision:.4f}")
    print(f"Fold {fold + 1} - Recall: {recall:.4f}")
    print(f"Fold {fold + 1} - F1 Score: {f1:.4f}")

    # Log de métricas en WandB
    wandb.log({
        f"Fold {fold + 1} Accuracy": accuracy,
        f"Fold {fold + 1} Precision": precision,
        f"Fold {fold + 1} Recall": recall,
        f"Fold {fold + 1} F1 Score": f1,
    })

# Guardar el modelo entrenado en un archivo
model_filename = "svm_model_Letters.pkl"
joblib.dump(svm_model, model_filename)
print(f"Modelo guardado como {model_filename}")

# Descargar el modelo (si estás en un entorno de notebook)
wandb.save(model_filename)

# Finalizar el experimento en WandB
wandb.finish()


wandb: Currently logged in as: andreu-mir (xisca). Use `wandb login --relogin` to force relogin


Fold 1/5
Fold 1 - Accuracy: 87.65%
Fold 1 - Precision: 0.8782
Fold 1 - Recall: 0.8765
Fold 1 - F1 Score: 0.8766
Fold 2/5
Fold 2 - Accuracy: 87.23%
Fold 2 - Precision: 0.8742
Fold 2 - Recall: 0.8723
Fold 2 - F1 Score: 0.8725
Fold 3/5
Fold 3 - Accuracy: 86.81%
Fold 3 - Precision: 0.8709
Fold 3 - Recall: 0.8681
Fold 3 - F1 Score: 0.8683
Fold 4/5
Fold 4 - Accuracy: 87.31%
Fold 4 - Precision: 0.8755
Fold 4 - Recall: 0.8731
Fold 4 - F1 Score: 0.8733
Fold 5/5
Fold 5 - Accuracy: 88.38%
Fold 5 - Precision: 0.8874
Fold 5 - Recall: 0.8838
Fold 5 - F1 Score: 0.8845
Modelo guardado como svm_model_Letters.pkl


Fold 1 Accuracy,▁
Fold 1 F1 Score,▁
Fold 1 Precision,▁
Fold 1 Recall,▁
Fold 2 Accuracy,▁
Fold 2 F1 Score,▁
Fold 2 Precision,▁
Fold 2 Recall,▁
Fold 3 Accuracy,▁
Fold 3 F1 Score,▁
Fold 3 Precision,▁
